In [15]:
import pandas as pd

df = pd.read_csv('test.tsv', sep='\t')

In [16]:
print(df['category'].unique())
# count number of each category
print(df['category'].value_counts())

['business' 'health' 'politics' 'sports' 'technology']
category
business      100
health        100
politics      100
sports        100
technology     22
Name: count, dtype: int64


In [20]:
from sklearn.preprocessing import LabelEncoder
clean_df = df.copy()

clean_df.drop('url', axis=1, inplace=True)
le = LabelEncoder()
clean_df['category'] = le.fit_transform(df['category'])

[0 1 2 3 4]


In [28]:
# Tokenize the text it is french
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-french-europeana-cased')
clean_df['headline'] = clean_df['headline'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

c:\Programmation\School\TPS\TAL\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\spoto\.cache\huggingface\hub\models--dbmdz--bert-base-french-europeana-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [24]:
clean_df.head()

,category,headline,text
0,0,"[101, 16007, 3600, 4487, 7140, 2546, 1024, 429...","Fondateur de L'africamobile à Dakar, Malick Di..."
1,0,"[101, 10364, 28940, 10448, 2139, 2053, 19908, ...","Un McDonald's à Tampa, en Floride, offre 50 do..."
2,0,"[101, 4421, 2015, 4241, 21164, 1024, 1005, 122...","Dans les rues animées de Benin City, au Nigéri..."
3,0,"[101, 8264, 20248, 21197, 19516, 1024, 4895, 4...","Par Zaria Gorvett * ""FEARFUL DISASTER"", titrai..."
4,0,"[101, 4078, 7505, 5737, 18941, 2015, 11703, 30...",Les survivants d'une attaque meurtrière contre...


In [29]:
from sklearn.model_selection import train_test_split
import torch

train_df, val_df = train_test_split(clean_df, test_size=0.2, random_state=42)

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        headline = torch.tensor(row['headline'])
        category = torch.tensor(row['category'])
        return headline, category
    
train_dataset = NewsDataset(train_df)
val_dataset = NewsDataset(val_df)

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()




ModuleNotFoundError: No module named 'torch'